## Imports

In [1]:
import pandas as pd
import plotly.express as px

import dash
from dash import dcc
from dash import html
import datetime
from dash.dependencies import Input, Output, State
import pandas_datareader.data as web

## Carregando dados dos valores das ações

In [2]:
# https://stooq.com/
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2021, 2, 15)

df = web.DataReader(['SVAC','JNJ','PFE','MRNA','BNTX'],
                    'stooq', start=start, end=end)

df = df.stack().reset_index()
df.columns = ['data', 'acao', 'close', 'high', 'low', 'open', 'volume']
df.head()

,data,acao,close,high,low,open,volume
0,2019-01-02,JNJ,114.9620,115.5310,113.6470,115.3150,8.480359e+06
1,2019-01-02,PFE,37.3441,37.5599,36.8123,37.2319,2.904233e+07
2,2019-01-02,MRNA,15.3300,15.4500,14.5100,14.9300,2.318094e+06
3,2019-01-03,JNJ,113.1380,115.4320,112.8140,115.3150,9.617137e+06
4,2019-01-03,PFE,36.2979,37.4203,36.2209,37.3343,3.132042e+07


In [3]:
cores = ["black", "red", "blue", "orange"]

In [4]:
acoes = df['acao'].unique()
acoes

array(['JNJ', 'PFE', 'MRNA', 'BNTX'], dtype=object)

## Iniciando Aplicação

In [5]:
app = dash.Dash()
app.title = 'Dashboard'

In [6]:
app.layout = html.Div(
    children = [
        html.H1(id='titulo-output', children='Valor ação covid-19', style={}),

        dcc.Dropdown(id='acoes-drop',
                     multi=True,
                     options=[{'label': x, 'value': x} for x in sorted(acoes)],
                     value=acoes,
                     style={'text-align': 'center'}),
        
        html.Br(),

        dcc.Graph(id='grafico-output', figure={}),

        dcc.RadioItems(id='cores-id', value='black', options=[{'label': x, 'value': x} for x in cores])
    ]
)

**Callback**

In [10]:
@app.callback(
    [Output('grafico-output', 'figure'), Output('titulo-output', 'style')],
    [Input(component_id='cores-id', component_property='value'),
    Input(component_id='acoes-drop', component_property='value')],
    prevent_initial_cal=False
)
def atualiza_grafico(cor_sel, acao_sel):
    
    if not len(acao_sel):
        return dash.no_update, {'color': cor_sel}
    else:
        df_tmp = df[df['acao'].isin(acao_sel)]
        fig = px.line(df_tmp, x='data', y='close', color='acao')

    return fig, {'color': cor_sel, 'text-align': 'center'}

In [11]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
